In [2]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import torch

accelerator = Accelerator()
device = accelerator.device

checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

In [3]:
from llmexp.explainer.mab_explainer import MABExplainer, MABTemplate
instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive', 'negative', or 'neutral', based on the overall tone and meaning of the sentence, if no enough information provided, respond with 'not clear' with an explanation."

template = MABTemplate(tokenizer, instruction)
explainer = MABExplainer(llm, tokenizer, template)

In [4]:
user_input = "Hello world! This is a test, or not a test. How are you?"
template_input = template.format(user_input) 
tokenized_template_input = tokenizer(template_input, return_tensors="pt").to(device)
# get input_ids and attention_mask
input_ids = tokenized_template_input.input_ids 
attention_mask = tokenized_template_input.attention_mask 

# get the output_ids and output_attention_mask
output = llm.generate(input_ids, attention_mask, max_new_tokens=100)
output_ids = output['input_ids']
output_attention_mask = output['attention_mask']

# get the response mask
response_mask = explainer.get_response_mask(attention_mask, output_attention_mask)

response_ids = output_ids[response_mask == 1]
response_texts = tokenizer.decode(response_ids[0], skip_special_tokens=False)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [5]:
response_texts

'neutral'

In [6]:
random_clips = explainer.random_clip_query_words(user_input, response_texts)

for clip in random_clips:
    print(clip)
    print("-"*100)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive', 'negative', or 'neutral', based on the overall tone and meaning of the sentence, if no enough information provided, respond with 'not clear' with an explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>

This a a How you ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

neutral
----------------------------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive', 'negative', or 'neutral', based on the overall tone and meaning of the sentence, if no enough information provided, respond with 'not clear' with an explanation.<|eot_id|>

In [2]:
from llmexp.explainer.mab_explainer import MultiLevelInputMapper
mapper = MultiLevelInputMapper()

In [3]:
mapper.process_text("Hello world! This is a test, or not a test. How are you?")

In [4]:
mapper.get_content(1,1)

'or not a test.'

In [5]:
# user_input = "The service at this restaurant was fantastic, and the staff were so friendly."
mapper.process_text(user_input) 

clipped_content_indices = mapper.sample_phrases(p=0.5)
# concatenate the content of the clipped_content_indices
clipped_content = " ".join([mapper.get_content(*i) for i in clipped_content_indices])


content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "user", 
            "content": clipped_content
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)

print(template)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive', 'negative', or 'neutral', based on the overall tone and meaning of the sentence, if no enough information provided, respond with 'not clear' with an explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>

Hello world!<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [6]:
inputs = tokenizer(template)
input_ids = torch.tensor(inputs["input_ids"]).unsqueeze(0).to(device)
attention_mask = torch.tensor(inputs["attention_mask"]).unsqueeze(0).to(device)
output = llm.generate(input_ids, attention_mask, max_new_tokens=100)
print(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input_ids': tensor([[128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,
           2696,     25,   6790,    220,   2366,     18,    198,  15724,   2696,
             25,    220,   1627,  10263,    220,   2366,     19,    271,   2127,
          56956,    279,  27065,    315,    279,   2768,  11914,    323,   6013,
            449,   1193,    832,   3492,     25,    364,  31587,    518,    364,
          43324,    518,    477,    364,  60668,    518,   3196,    389,    279,
           8244,  16630,    323,   7438,    315,    279,  11914,     11,    422,
            912,   3403,   2038,   3984,     11,   6013,    449,    364,   1962,
           2867,      6,    449,    459,  16540,     13, 128009, 128006,    882,
         128007,    271,   9906,   1917,      0, 128009, 128006,  78191, 128007,
            271,  60668, 128009]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 

In [7]:
output_ids = output['input_ids']
output_attention_mask = output['attention_mask']

output_texts = tokenizer.decode(output_ids[0], skip_special_tokens=False)
print(output_texts)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive', 'negative', or 'neutral', based on the overall tone and meaning of the sentence, if no enough information provided, respond with 'not clear' with an explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>

Hello world!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

neutral<|eot_id|>


In [8]:
from llmexp.explainer.mab_explainer import MABExplainer
explainer = MABExplainer(llm, tokenizer, device)


In [9]:
response_mask = explainer.get_response_mask(attention_mask, output_attention_mask)

In [10]:
results = explainer.get_response_logits(output_ids, output_attention_mask, response_mask)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [11]:
explainer.get_response_logits_mean(output_ids, output_attention_mask, response_mask)

tensor([20.2944], device='cuda:0')